# Experiments!

## Imports

In [2]:
import os
import torch
import numpy as np
import time
from torch.utils.data import Dataset, DataLoader
from torch import cuda

## Loading datasets

In [3]:
!rm -rf datasets
!mkdir datasets
!git clone https://github.com/ncg-task/training-data.git datasets/train
!git clone https://github.com/ncg-task/trial-data.git datasets/validation
!git clone https://github.com/ncg-task/test-data datasets/test

Cloning into 'datasets/train'...
remote: Enumerating objects: 3083, done.
remote: Counting objects: 100% (3083/3083), done.
remote: Compressing objects: 100% (2728/2728), done.
remote: Total 6864 (delta 567), reused 2504 (delta 279), pack-reused 3781
Receiving objects: 100% (6864/6864), 157.36 MiB | 20.74 MiB/s, done.
Resolving deltas: 100% (660/660), done.
Cloning into 'datasets/validation'...
remote: Enumerating objects: 1636, done.
remote: Counting objects: 100% (1636/1636), done.
remote: Compressing objects: 100% (1438/1438), done.
remote: Total 1636 (delta 580), reused 1224 (delta 170), pack-reused 0
Receiving objects: 100% (1636/1636), 27.88 MiB | 20.85 MiB/s, done.
Resolving deltas: 100% (580/580), done.
Cloning into 'datasets/test'...
remote: Enumerating objects: 2508, done.
remote: Total 2508 (delta 0), reused 0 (delta 0), pack-reused 2508
Receiving objects: 100% (2508/2508), 215.28 MiB | 18.43 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Checking out files: 100% (2060/2

In [22]:
input_dir = "datasets/train/"

device = 'cuda' if cuda.is_available() else 'cpu'
list_of_folders = ["query_wellformedness", "passage_re-ranking", "part-of-speech_tagging", 
         "sentence_compression", "sentiment_analysis", "temporal_information_extraction", 
         "phrase_grounding", "text_generation", "text-to-speech_synthesis", 
         "smile_recognition", "topic_models", "question_generation", 
         "relation_extraction", "paraphrase_generation", "question_similarity", 
         "question_answering", "sentence_classification", "prosody_prediction", 
         "semantic_role_labeling", "text_summarization", "semantic_parsing",
         "sarcasm_detection", "natural_language_inference", "negation_scope_resolution"]
input_stanza_list = []
input_stanza_len = []
input_sent_num_list = []
file_name_list = []
for fls in list_of_folders:
  count=0
  for i in os.listdir(input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None,map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        input_stanza_len.append(len(stanza_lines_list))
        input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        input_sent_num_list.append(sentence_num_list)
    file_name_list.append(fls + '/' + str(i))
  print("Training complete",fls, count)

Training complete query_wellformedness 1
Training complete passage_re-ranking 2
Training complete part-of-speech_tagging 8
Training complete sentence_compression 4
Training complete sentiment_analysis 52
Training complete temporal_information_extraction 2
Training complete phrase_grounding 1
Training complete text_generation 6
Training complete text-to-speech_synthesis 3
Training complete smile_recognition 1
Training complete topic_models 1
Training complete question_generation 2
Training complete relation_extraction 14
Training complete paraphrase_generation 2
Training complete question_similarity 1
Training complete question_answering 6
Training complete sentence_classification 3
Training complete prosody_prediction 1
Training complete semantic_role_labeling 5
Training complete text_summarization 15
Training complete semantic_parsing 3
Training complete sarcasm_detection 2
Training complete natural_language_inference 101
Training complete negation_scope_resolution 1


In [23]:
val_input_dir = "datasets/validation/"
val_list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
         "relation-classification", "text-classification"]
val_input_stanza_list = []
val_input_sent_num_list = []
val_file_name_list = []
val_input_stanza_len = []
for fls in val_list_of_folders:
  count=0
  for i in os.listdir(val_input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(val_input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        val_input_stanza_len.append(len(stanza_lines_list))
        val_input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        val_input_sent_num_list.append(sentence_num_list)
    val_file_name_list.append(fls + '/' + str(i))
  print("Valid complete",fls,count)

Valid complete machine-translation 10
Valid complete named-entity-recognition 10
Valid complete question-answering 10
Valid complete relation-classification 10
Valid complete text-classification 10


In [24]:
input_sent_num_list = [[int(s) for s in sublist] for sublist in input_sent_num_list] # convert sentence list string to integer
input_sent_num_list = [list(set(x)) for x in input_sent_num_list]

#### conversion for Valid set
val_input_sent_num_list = [[int(s) for s in sublist] for sublist in val_input_sent_num_list] # convert sentence list string to integer
val_input_sent_num_list = [list(set(x)) for x in val_input_sent_num_list]

In [25]:
#### Ground truth label formation for classification
#### Train set
multihot_input_sent = []
for i in range(len(input_stanza_list)):
  temp =[0]*input_stanza_len[i]
  for j in range(len(input_sent_num_list[i])):
    t1 = input_sent_num_list[i][j] -1
    temp[t1] = 1
  multihot_input_sent.append(temp)
#### valid set
val_multihot_input_sent = []
for i in range(len(val_input_stanza_list)):
  temp =[0]*val_input_stanza_len[i]
  for j in range(len(val_input_sent_num_list[i])):
    t1 = val_input_sent_num_list[i][j] -1
    temp[t1] = 1
  val_multihot_input_sent.append(temp)

In [26]:
## Function for flattening 2d list to 1d list
from collections import Iterable
def flatten(lis):
  for item in lis:
    if isinstance(item, Iterable) and not isinstance(item, str):
      for x in flatten(item):
        yield x
    else:        
      yield item

In [30]:
#### Flatten the list and use heuristic to remove vague sentences
#### Train set
from collections import Counter

train_sentences = list(flatten(input_stanza_list))
train_label = list(flatten(multihot_input_sent))

train_tuple = list(set((zip(train_sentences,train_label))))
print(len(train_tuple))
train_in_sentences = []
train_sent_label = []
for stan,lab in train_tuple:
  if len(stan) >4:
    train_in_sentences.append(stan)
    train_sent_label.append(lab)

valid_sentences = list(flatten(val_input_stanza_list))
valid_label = list(flatten(val_multihot_input_sent))

valid_tuple = list(set((zip(valid_sentences,valid_label))))
valid_in_sentences = []
valid_sent_label = []
for stan,lab in valid_tuple:
  if len(stan) >4:
    valid_in_sentences.append(stan)
    valid_sent_label.append(lab)

print("train size",len(train_in_sentences),len(train_sent_label))
print("valid size",len(valid_in_sentences),len(valid_sent_label))

51538
train size 51296 51296
valid size 10681 10681


In [33]:
label0_sent = [i for i, j in zip(train_in_sentences, train_sent_label) if j == 0]
label1_sent = [i for i, j in zip(train_in_sentences, train_sent_label) if j == 1]

In [34]:
print(len(label0_sent), len(label1_sent))

46271 5025


In [35]:
train_tuple = list(set((zip(train_in_sentences,train_sent_label))))
train_sum_sent = 3 * [stan for stan,label in train_tuple if label==1]
train_nonsum_sent = [stan for stan,label in train_tuple if label==0]
train_in_sentences = train_sum_sent + train_nonsum_sent
train_sent_label = len(train_sum_sent)*[1] + len(train_nonsum_sent)*[0]
print("Oversampled train size",len(train_in_sentences),len(train_sent_label))

Oversampled train size 61346 61346


In [36]:
label0_sent = [i for i, j in zip(train_in_sentences, train_sent_label) if j == 0]
label1_sent = [i for i, j in zip(train_in_sentences, train_sent_label) if j == 1]

In [38]:
print(len(label0_sent), len(label1_sent))

46271 15075


In [39]:
!pip install -q transformers sentencepiece

In [40]:
from transformers import *

# Load the BERT tokenizer.
print('Loading SciBERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

Loading SciBERT tokenizer...


In [41]:
train_pairs = [(i, j) for i, j in zip(train_in_sentences, train_sent_label)]
validation_pairs = [(i, j) for i, j in zip(valid_in_sentences, valid_sent_label)]

In [42]:
train_pairs[0]

('the gated recurrent unit variant of lstm is used throughout our model .', 1)

In [53]:
max_len = 0
less256 = 0
less512 = 0
less128 = 0
token_lengths = []

# For every sentence...
for sent in train_in_sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))
    if len(input_ids) < 512:
        less512 += 1
    
    if len(input_ids) < 256:
        less256 += 1
    
    if len(input_ids) < 128:
        less128 += 1
    
    token_lengths.append(len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  401


In [54]:
len(token_lengths)

61346

In [50]:
print(less512, less256, less128)

61346 61344 61287


In [52]:
61287 / 61346

0.9990382421021745

In [44]:
train_input_ids = []
train_attention_masks = []
train_input_ids = []
train_lengths = []

# For every sentence...
for sent in train_in_sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        truncation=True,
                        return_length=True,
                        padding = "max_length",
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    train_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    train_attention_masks.append(encoded_dict['attention_mask'])
    train_lengths.append(encoded_dict['length'])
    

# Convert the lists into tensors.
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(train_sent_label)
train_lengths = torch.tensor(train_lengths)

In [48]:
len(train_input_ids[0])

256

In [46]:
sum(1 for i in train_input_ids if len(i) < 256)

0

In [17]:
validation_input_ids = []
validation_attention_masks = []
validation_input_ids = []
validation_lengths = []

# For every sentence...
for sent in valid_in_sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        truncation=True,
                        return_length=True,
                        padding = "max_length",
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    validation_input_ids.append(encoded_dict['input_ids'])
    validation_lengths.append(encoded_dict['length'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    validation_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
validation_input_ids = torch.cat(validation_input_ids, dim=0)
validation_attention_masks = torch.cat(validation_attention_masks, dim=0)
validation_labels = torch.tensor(valid_sent_label)
validation_lengths = torch.tensor(validation_lengths)

In [18]:
print(len(validation_input_ids), len(validation_attention_masks), len(validation_labels), len(validation_lengths))

10681 10681 10681 10681


In [19]:
from torch.utils.data import TensorDataset

In [20]:
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_lengths, train_labels)
validation_dataset = TensorDataset(validation_input_ids, validation_attention_masks, validation_lengths, validation_labels)

In [21]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            validation_dataset, # The validation samples.
            sampler = SequentialSampler(validation_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [21]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = AutoModelForSequenceClassification.from_pretrained('allenai/scibert_scivocab_uncased', output_attentions=False, output_hidden_states = False, num_labels = 2)
# model = BertForSequenceClassification.from_pretrained(
#     "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = 2, # The number of output labels--2 for binary classification.
#                     # You can increase this for multi-class tasks.   
#     output_attentions = False, # Whether the model returns attentions weights.
#     output_hidden_states = False, # Whether the model returns all hidden-states.
# )

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [22]:
# Create a SciBERT + Bi-LSTM model for binary classification
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.bert = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased',  output_hidden_states=True)
#         self.lstm = torch.nn.LSTM(768, 400, num_layers=2, batch_first = True, bidirectional=True)
#         self.l1 = torch.nn.Linear(768, 800)  
#         self.l2 = torch.nn.Dropout(0.1)
#         self.l3 = torch.nn.Linear( 800, 200)
#         self.l4 = torch.nn.Linear(200,2)
        
        self.l1 = torch.nn.Sequential(torch.nn.Dropout(0.1), torch.nn.Linear(768, 800), torch.nn.Dropout(0.1), torch.nn.ReLU(), torch.nn.Linear(800, 200), torch.nn.ReLU(), torch.nn.Linear(200, 2))
    
    def forward(self, ids, mask, token_type_ids):
        encoded_layers = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)[2]
        scibert_hidden_layer = encoded_layers[12]
        scibert_hidden_layer = scibert_hidden_layer[:,0,:].view(-1,768)

#         enc_hiddens, (last_hidden, last_cell) = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(scibert_hidden_layer, lengths.cpu(), batch_first=True, enforce_sorted=False)) #enforce_sorted=False  #pack_padded_sequence(data and batch_sizes
#         output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)  # (batch_size, 2*hidden_size)
#         output_hidden = self.l1(scibert_hidden_layer)
#         output_hidden = self.l2(output_hidden)
#         output_2 = self.l3(output_hidden)
#         output_3 = torch.nn.ReLU()(output_2)
#         output_4 = self.l4(output_3)
        
        output = self.l1(scibert_hidden_layer)
        return output

In [23]:
model = NeuralNetwork()
model.cuda()

NeuralNetwork(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [24]:
label0_sent = [i for i, j in zip(train_sentences, train_labels) if j == 0]
label1_sent = [i for i, j in zip(train_sentences, train_labels) if j == 1]

In [25]:
print(len(label0_sent), len(label1_sent))

40126 15075


In [26]:
c_weights = [len(label1_sent), len(label0_sent)]
c_weights = [i/sum(c_weights) for i in c_weights]

c_weights = torch.tensor(c_weights)
c_weights

tensor([0.2731, 0.7269])

In [27]:
CELoss = torch.nn.CrossEntropyLoss(weight = c_weights.cuda())

In [1]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

NameError: name 'model' is not defined

In [29]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [30]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [31]:
import numpy as np
from sklearn.metrics import f1_score

# Function to calculate the accuracy of our predictions vs labels
def results(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    accuracy =  np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    f1 = f1_score(labels, pred_flat)
    return accuracy, f1

In [32]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [33]:
import gc

gc.collect()
import torch

torch.cuda.empty_cache()

In [55]:
# for batch in train_dataloader:
#   break

# ids = batch[0].to(device)
# masks = batch[1].to(device)
# lens = batch[2].to(device)
# labels = batch[3].to(device)

In [56]:
# outputs = model(ids, masks, None, lens)

In [57]:
# loss = CELoss(outputs, labels)

In [58]:
loss

tensor(0.6145, device='cuda:0', grad_fn=<NllLossBackward>)

In [59]:
# model = NeuralNetwork()
# model.load_state_dict(torch.load("/content/drive/MyDrive/NLP/Fancy-SciBERT-Oversampling/trained_model.bin"))
# model.cuda()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/NLP/Fancy-SciBERT-Oversampling/trained_model.bin'

In [72]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, 2):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, 2))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 50 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_lengths = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        outputs = model(b_input_ids, 
                       b_input_mask,
                       None)

        loss = CELoss(outputs, b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # # ========================================
    # #               Validation
    # # ========================================
    # # After the completion of each training epoch, measure our performance on
    # # our validation set.

    # print("")
    # print("Running Validation...")

    # t0 = time.time()

    # # Put the model in evaluation mode--the dropout layers behave differently
    # # during evaluation.
    # model.eval()

    # # Tracking variables 
    # total_eval_accuracy = 0
    # total_f1_score = 0
    # total_eval_loss = 0
    # nb_eval_steps = 0

    # # Evaluate data for one epoch
    # for batch in validation_dataloader:
        
    #     # Unpack this training batch from our dataloader. 
    #     #
    #     # As we unpack the batch, we'll also copy each tensor to the GPU using 
    #     # the `to` method.
    #     #
    #     # `batch` contains three pytorch tensors:
    #     #   [0]: input ids 
    #     #   [1]: attention masks
    #     #   [2]: labels 
    #     b_input_ids = batch[0].to(device)
    #     b_input_mask = batch[1].to(device)
    #     b_labels = batch[2].to(device)
        
    #     # Tell pytorch not to bother with constructing the compute graph during
    #     # the forward pass, since this is only needed for backprop (training).
    #     with torch.no_grad():        

    #         # Forward pass, calculate logit predictions.
    #         # token_type_ids is the same as the "segment ids", which 
    #         # differentiates sentence 1 and 2 in 2-sentence tasks.
    #         result = model(b_input_ids, 
    #                        token_type_ids=None, 
    #                        attention_mask=b_input_mask,
    #                        labels=b_labels,
    #                        return_dict=True)

    #     # Get the loss and "logits" output by the model. The "logits" are the 
    #     # output values prior to applying an activation function like the 
    #     # softmax.
    #     loss = result.loss
    #     logits = result.logits
            
    #     # Accumulate the validation loss.
    #     total_eval_loss += loss.item()

    #     # Move logits and labels to CPU
    #     logits = logits.detach().cpu().numpy()
    #     label_ids = b_labels.to('cpu').numpy()

    #     # Calculate the accuracy for this batch of test sentences, and
    #     # accumulate it over all batches.
    #     total_eval_accuracy += results(logits, label_ids)[0]
    #     total_f1_score += results(logits, label_ids)[1]
        

    # # Report the final accuracy for this validation run.
    # avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    # avg_f1 = total_f1_score / len(validation_dataloader)
    # print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # print("  F! score: {0:.2f}".format(avg_f1))

    # # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # # Measure how long the validation run took.
    # validation_time = format_time(time.time() - t0)
    
    # print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    # print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Training Time': training_time,
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...
  Batch    50  of  1,918.    Elapsed: 0:01:26.
  Batch   100  of  1,918.    Elapsed: 0:02:53.
  Batch   150  of  1,918.    Elapsed: 0:04:19.
  Batch   200  of  1,918.    Elapsed: 0:05:45.
  Batch   250  of  1,918.    Elapsed: 0:07:12.
  Batch   300  of  1,918.    Elapsed: 0:08:38.
  Batch   350  of  1,918.    Elapsed: 0:10:04.
  Batch   400  of  1,918.    Elapsed: 0:11:31.
  Batch   450  of  1,918.    Elapsed: 0:12:58.
  Batch   500  of  1,918.    Elapsed: 0:14:25.
  Batch   550  of  1,918.    Elapsed: 0:15:52.
  Batch   600  of  1,918.    Elapsed: 0:17:18.
  Batch   650  of  1,918.    Elapsed: 0:18:44.
  Batch   700  of  1,918.    Elapsed: 0:20:11.
  Batch   750  of  1,918.    Elapsed: 0:21:36.
  Batch   800  of  1,918.    Elapsed: 0:23:02.
  Batch   850  of  1,918.    Elapsed: 0:24:29.
  Batch   900  of  1,918.    Elapsed: 0:25:55.
  Batch   950  of  1,918.    Elapsed: 0:27:20.
  Batch 1,000  of  1,918.    Elapsed: 0:28:44.
  Batch 1,050  of

KeyboardInterrupt: 

In [73]:
torch.save(model.state_dict(), "./NotSoFancy-SciBERT/pretrained_model.bin")
tokenizer.save_pretrained("./NotSoFancy-SciBERT/")

('./NotSoFancy-SciBERT/tokenizer_config.json',
 './NotSoFancy-SciBERT/special_tokens_map.json',
 './NotSoFancy-SciBERT/vocab.txt',
 './NotSoFancy-SciBERT/added_tokens.json')

In [66]:
for batch in validation_dataloader:
  break

In [71]:
model.eval()
predictions, true_labels = [], []
ids = batch[0].to(device)
mask = batch[1].to(device)
lens = batch[2].to(device)
labels = batch[3].to(device)

with torch.no_grad():
  outputs = model(ids, mask, None, lens)


In [75]:
output_idx.cpu().numpy()

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1])

In [55]:
!ls /content/drive/MyDrive/Vanilla-SciBERT-Nooversampling

special_tokens_map.json  tokenizer_config.json	vocab.txt


In [57]:
print('Predicting labels for {:,} test sentences...'.format(len(validation_dataloader) * batch_size))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for idx, batch in enumerate(validation_dataloader):
  if idx % 10 == 0:
    print(f"Done with {idx} of {len(validation_dataloader)}")
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_lengths, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      outputs = model(b_input_ids, 
                       b_input_mask,
                       None)
      _, labels = torch.max(outputs, dim = 1)

  logits = labels.cpu().numpy()
  label_ids = b_labels.cpu().numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 10,688 test sentences...
Done with 0 of 334
Done with 10 of 334
Done with 20 of 334
Done with 30 of 334
Done with 40 of 334
Done with 50 of 334
Done with 60 of 334
Done with 70 of 334
Done with 80 of 334
Done with 90 of 334
Done with 100 of 334
Done with 110 of 334
Done with 120 of 334
Done with 130 of 334
Done with 140 of 334
Done with 150 of 334
Done with 160 of 334
Done with 170 of 334
Done with 180 of 334
Done with 190 of 334
Done with 200 of 334
Done with 210 of 334
Done with 220 of 334
Done with 230 of 334
Done with 240 of 334
Done with 250 of 334
Done with 260 of 334
Done with 270 of 334
Done with 280 of 334
Done with 290 of 334
Done with 300 of 334
Done with 310 of 334
Done with 320 of 334
Done with 330 of 334
    DONE.


In [58]:
from sklearn.metrics import matthews_corrcoef

# Combine the results across all batches. 
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
# flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('Total MCC: %.3f' % mcc)

Total MCC: 0.366


In [59]:
from sklearn.metrics import classification_report
    
print(classification_report(flat_true_labels, flat_predictions))

              precision    recall  f1-score   support

           0       0.95      0.91      0.93      9685
           1       0.37      0.52      0.43       996

    accuracy                           0.87     10681
   macro avg       0.66      0.71      0.68     10681
weighted avg       0.89      0.87      0.88     10681



In [60]:
!ls datasets/test

constituency_parsing	 document_classification  hypernym_discovery
coreference_resolution	 entity_linking		  natural_language_inference
data-to-text_generation  face_alignment		  README.md
dependency_parsing	 face_detection


In [ ]:
sentence_file_id = {}

In [77]:
test_input_dir = "datasets/test/"
test_list_of_folders = ["constituency_parsing", "document_classification", "hypernym_discovery",
         "coreference_resolution", "entity_linking", "natural_language_inference", "data-to-text_generation", "face_alignment", "dependency_parsing", "face_detection"]
test_stanza_list = []
test_sent_num_list = []
test_file_name_list = []
test_stanza_len = []

for fls in test_list_of_folders:
  count=0
  for i in os.listdir(test_input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(test_input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        test_stanza_len.append(len(stanza_lines_list))
        test_stanza_list.append(stanza_lines_list)

      if files.endswith("sentences.txt"):
        sentence_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        test_sent_num_list.append(sentence_num_list)
    test_file_name_list.append(fls + '/' + str(i))
  print("Test complete",fls,count)

Test complete constituency_parsing 9
Test complete document_classification 21
Test complete hypernym_discovery 9
Test complete coreference_resolution 10
Test complete entity_linking 17
Test complete natural_language_inference 32
Test complete data-to-text_generation 7
Test complete face_alignment 19
Test complete dependency_parsing 9
Test complete face_detection 22


In [78]:
test_sent_num_list = [[int(s) for s in sublist] for sublist in test_sent_num_list]
test_sent_num_list = [list(set(x)) for x in test_sent_num_list]

In [84]:
test_stanza_list[0]

['title',
 'constituency parsing with a self - attentive encoder',
 'abstract',
 'we demonstrate that replacing an lstm encoder with a self - attentive architecture can lead to improvements to a state - of the - art discriminative constituency parser .',
 'the use of attention makes explicit the manner in which information is propagated between different locations in the sentence , which we use to both analyze our model and propose potential improvements .',
 'for example , we find that separating positional and content information in the encoder can lead to improved parsing accuracy .',
 'additionally , we evaluate different approaches for lexical representation .',
 'our parser achieves new state - of the - art results for single models trained on the penn treebank : 93.55 f1 without the use of any external data , and 95.13 f1 when using pre-trained word representations .',
 'our parser also outperforms the previous best - published accuracy figures on 8 of the 9 languages in the spm

In [79]:
multihot_test_sent = []

for i in range(len(test_stanza_list)):
  temp = [0] * test_stanza_len[i]
  for j in range(len(test_sent_num_list[i])):
    t1 = test_sent_num_list[i][j] - 1
    temp[t1] = 1
  multihot_test_sent.append(temp)

In [80]:
#### Flatten the list and use heuristic to remove vague sentences
#### Train set
from collections import Counter

test_sentences = list(flatten(test_stanza_list))
test_label = list(flatten(multihot_test_sent))

test_tuple = list(set((zip(test_sentences, test_label))))
test_sentences = []
test_sent_label = []
for stan,lab in test_tuple:
  if len(stan) >4:
    test_sentences.append(stan)
    test_sent_label.append(lab)


print("test size",len(test_sentences),len(test_sent_label))

test size 31619 31619


In [81]:
test_attention_masks = []
test_ids = []
test_lens = []

# For every sentence...
for sent in test_sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        truncation=True,
                        padding = "max_length",
                        return_length=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    test_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    test_attention_masks.append(encoded_dict['attention_mask'])
    test_lens.append(encoded_dict['length'])

# Convert the lists into tensors.
test_ids = torch.cat(test_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
test_sent_label = torch.tensor(test_sent_label)
test_lens = torch.tensor(test_lens)

In [82]:
print(len(test_ids), len(test_attention_masks), len(test_sent_label), len(test_sentences))

31619 31619 31619 31619


In [76]:
test_dataset = TensorDataset(test_ids, test_attention_masks, test_lens, test_sent_label)

AttributeError: 'list' object has no attribute 'size'

In [68]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
test_dataloader = DataLoader(
            test_dataset,  # The training samples.
            sampler = RandomSampler(test_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

In [69]:
print('Predicting labels for {:,} test sentences...'.format(len(test_dataloader) * batch_size))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for idx, batch in enumerate(test_dataloader):
  if idx % 10 == 0:
    print(f"Done with {idx} of {len(test_dataloader)}")
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_lengths, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      outputs = model(b_input_ids, 
                       b_input_mask,
                       None)
      _, labels = torch.max(outputs, dim = 1)

  logits = labels.cpu().numpy()
  label_ids = b_labels.cpu().numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('DONE.')

Predicting labels for 31,648 test sentences...
Done with 0 of 989
Done with 10 of 989
Done with 20 of 989
Done with 30 of 989
Done with 40 of 989
Done with 50 of 989
Done with 60 of 989
Done with 70 of 989
Done with 80 of 989
Done with 90 of 989
Done with 100 of 989
Done with 110 of 989
Done with 120 of 989
Done with 130 of 989
Done with 140 of 989
Done with 150 of 989
Done with 160 of 989
Done with 170 of 989
Done with 180 of 989
Done with 190 of 989
Done with 200 of 989
Done with 210 of 989
Done with 220 of 989
Done with 230 of 989
Done with 240 of 989
Done with 250 of 989
Done with 260 of 989
Done with 270 of 989
Done with 280 of 989
Done with 290 of 989
Done with 300 of 989
Done with 310 of 989
Done with 320 of 989
Done with 330 of 989
Done with 340 of 989
Done with 350 of 989
Done with 360 of 989
Done with 370 of 989
Done with 380 of 989
Done with 390 of 989
Done with 400 of 989
Done with 410 of 989
Done with 420 of 989
Done with 430 of 989
Done with 440 of 989
Done with 450 of 98

In [70]:
from sklearn.metrics import matthews_corrcoef

# Combine the results across all batches. 
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
# flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('Total MCC: %.3f' % mcc)

Total MCC: 0.371


In [71]:
from sklearn.metrics import classification_report

print(classification_report(flat_true_labels, flat_predictions))

              precision    recall  f1-score   support

           0       0.95      0.92      0.94     28957
           1       0.36      0.52      0.43      2662

    accuracy                           0.88     31619
   macro avg       0.66      0.72      0.68     31619
weighted avg       0.90      0.88      0.89     31619



In [85]:
import os

In [86]:
model

NeuralNetwork(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [101]:
from shutil import copyfile

test_input_dir = "datasets/test/"
output_dir = "datasets/output_subtask_A/"

test_list_of_folders = [f.path.split("/")[-1] for f in os.scandir(test_input_dir) if f.is_dir()]

test_input_stanza_list = []
test_file_name_list = []
test_input_stanza_len = []

MAX_LEN=100

for fls in test_list_of_folders:
  for i in os.listdir(test_input_dir + fls + '/'):
    count=count+1
    output_info_folder = os.path.join(output_dir,fls,str(i))
    os.makedirs(output_info_folder)
    os.makedirs(os.path.join(output_info_folder, "triples"))
    fpath = os.path.join(output_info_folder, "entities.txt")
    f = open(fpath, "w")
    f.close()
    for files in os.listdir(test_input_dir + fls + '/' + str(i)):
#       if files.endswith("Grobid-out.txt"):
#         copyfile(test_input_dir + fls + '/' + str(i) + '/' + files, output_dir + fls + '/' + str(i) + '/' + files)
      
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
#         copyfile(test_input_dir + fls + '/' + str(i) + '/' + files, output_dir + fls + '/' + str(i) + '/' + files)
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        test_input_stanza_len.append(len(stanza_lines_list))
        test_input_stanza_list.append(stanza_lines_list)
    
    print("size of file",len(stanza_lines_list),fls+'/'+str(i))
    with torch.no_grad():
      output_file=open(os.path.join(output_info_folder, "sentences.txt"),"a")
      if stanza_lines_list[1] != 'abstract':
        output_file.write(str(2) +"\n")
      for t8 in range(2,len(stanza_lines_list)):
        if (stanza_lines_list[t8]=='conclusion') or (stanza_lines_list[t8]=='conclusions'):
          break
        test_input = tokenizer.encode_plus(stanza_lines_list[t8],
                                           add_special_tokens=True,
                                           max_length=256,
                                           truncation=True,
                                           return_length=True,
                                           padding="max_length",
                                           return_attention_mask=True,   # Construct attn. masks.
                                           return_tensors='pt',     # Return pytorch tensors.
                                            )
        test_ids = test_input['input_ids'].to(device, dtype = torch.long)
        test_mask = test_input['attention_mask'].to(device, dtype = torch.long)
        test_token_type_ids = test_input['token_type_ids'].to(device, dtype = torch.long)
        lengths = test_input['length']
        outputs = model(test_ids, test_mask, test_token_type_ids)
        _, output_idx = torch.max(outputs, dim =1)
        t_output_class = output_idx.cpu().detach().numpy().tolist()
        output_file=open(os.path.join(output_info_folder, "sentences.txt"),"a")
        #writing the sentences.txt file showing contribution sentences
        if t_output_class[0] == 1:
          output_file.write(str(t8+1) +"\n")
    test_file_name_list.append(fls + '/' + str(i))
  print("Test complete",fls,count)

size of file 248 coreference_resolution/8
size of file 127 coreference_resolution/6
size of file 129 coreference_resolution/7
size of file 134 coreference_resolution/3
size of file 256 coreference_resolution/0
size of file 235 coreference_resolution/5
size of file 118 coreference_resolution/9
size of file 150 coreference_resolution/2
size of file 240 coreference_resolution/4
size of file 147 coreference_resolution/1
Test complete coreference_resolution 317
size of file 220 natural_language_inference/22
size of file 201 natural_language_inference/18
size of file 341 natural_language_inference/31
size of file 120 natural_language_inference/8
size of file 244 natural_language_inference/28
size of file 404 natural_language_inference/17
size of file 257 natural_language_inference/25
size of file 152 natural_language_inference/27
size of file 138 natural_language_inference/6
size of file 221 natural_language_inference/21
size of file 205 natural_language_inference/12
size of file 285 natural